In [1]:
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta http-equiv="X-UA-Compatible" content="ie=edge">
  <title>Bellybutton Biodiversity</title>
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
</head>

<body>

  <div class="container">
    <div class="row">
      <div class="col-md-12 jumbotron text-center">
        <h1>Belly Button Biodiversity Dashboard</h1>
        <p>Use the interactive charts below to explore the dataset</p>
      </div>
    </div>
    <div class="row">
      <div class="col-md-2">
        <div class="well">
          <h5>Test Subject ID No.:</h5>
          <select id="selDataset" onchange="optionChanged(this.value)"></select>
        </div>
        <div class="panel panel-primary">
          <div class="panel-heading">
            <h3 class="panel-title">Demographic Info</h3>
          </div>
          <div id="sample-metadata" class="panel-body"></div>
        </div>
      </div>
      <div class="col-md-5">
        <div id="bar"></div>
      </div>
      <div class="col-md-5">
        <div id="gauge"></div>
      </div>
    </div>
    <div class="row">
      <div class="col-md-12">
        <div id="bubble"></div>
      </div>
    </div>
  </div>

  <script src="https://d3js.org/d3.v7.min.js"></script>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
  <script src="./static/js/app.js"></script>
  <script src="./static/js/bonus.js"></script>

</body>

</html>


SyntaxError: invalid syntax (2900496849.py, line 1)

In [2]:


import requests
import plotly.graph_objs as go
from IPython.display import display, HTML

# Read the samples.json data using requests
response = requests.get("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1/14-Interactive-Web-Visualizations/02-Homework/samples.json")
data = response.json()

# Assuming the data contains a "samples" array
samples = data['samples']

# Assuming you want to work with the first sample in the array
first_sample = samples[0]

# Assuming you want to create a bar chart using the "otu_ids", "otu_labels", and "sample_values" from the first sample
trace = go.Bar(
    x=first_sample['otu_ids'][:10][::-1],
    y=first_sample['otu_labels'][:10][::-1],
    text=first_sample['otu_labels'][:10][::-1],
    orientation='h'
)

layout = go.Layout(
    title="Top 10 OTUs Found in First Sample",
    xaxis=dict(title="OTU Count"),
    yaxis=dict(title="OTU IDs", type="category")
)

fig = go.Figure(data=[trace], layout=layout)
display(HTML(fig.to_html()))



ConnectionError: HTTPSConnectionPool(host='2u-data-curriculum-team.s3.amazonaws.com', port=443): Max retries exceeded with url: /dataviz-classroom/v1.1/14-Interactive-Web-Visualizations/02-Homework/samples.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026F9D453970>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [3]:
# Create a function to update the chart based on the selected sample
def update_chart(change):
    selected_sample_id = change['new']  # Change this line
    selected_sample = next((sample for sample in samples if sample['id'] == selected_sample_id), None)
    
    if selected_sample:
        trace = go.Bar(
            x=selected_sample['sample_values'][:10][::-1],
            y=[f'OTU {otu_id}' for otu_id in selected_sample['otu_ids'][:10][::-1]],
            text=selected_sample['otu_labels'][:10][::-1],
            orientation='h'
        )

        layout = go.Layout(
            title=f"Top 10 OTUs Found in Sample {selected_sample_id}",
            xaxis=dict(title="Sample Value"),
            yaxis=dict(title="OTU IDs", type="category")
        )

        fig = go.Figure(data=[trace], layout=layout)
        display(HTML(fig.to_html()))
    else:
        print("Selected sample not found!")

# Link the dropdown to the update function
dropdown.observe(update_chart, names='value')

# Display the dropdown and an initial chart
initial_sample_id = samples[0]['id']
update_chart({'new': initial_sample_id})

# Display the dropdown
display(dropdown)




NameError: name 'dropdown' is not defined

In [4]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import Dropdown, VBox, Output
from IPython.display import display

# Sample data
df = pd.DataFrame({
    "otu_ids": ["otu_1", "otu_2", "otu_3", "otu_4", "otu_5", "otu_6", "otu_7", "otu_8", "otu_9", "otu_10"],
    "sample_values": [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
    "otu_labels": ["OTU 1", "OTU 2", "OTU 3", "OTU 4", "OTU 5", "OTU 6", "OTU 7", "OTU 8", "OTU 9", "OTU 10"]
})

# Create a function to update the chart based on the selected option
def update_chart(change):
    selected_option = change.new
    if selected_option == "Top 10 OTUs":
        new_df = df.head(10)
    else:
        new_df = df
    with chart_output:
        fig = go.Figure(data=[go.Bar(x=new_df["sample_values"], y=new_df["otu_ids"], orientation="h", text=new_df["otu_labels"])])
        fig.update_layout(title=selected_option)
        fig.show()

# Create a Dropdown widget
dropdown_menu = [
    "Top 10 OTUs",
    "All OTUs"
]
dropdown = Dropdown(options=dropdown_menu, value="Top 10 OTUs", description="Select OTUs:")

# Create an output widget to display the chart
chart_output = Output()

# Attach the update function to the dropdown's value trait
dropdown.observe(update_chart, names="value")

# Display the dropdown and chart
display(dropdown, chart_output)



Dropdown(description='Select OTUs:', options=('Top 10 OTUs', 'All OTUs'), value='Top 10 OTUs')

Output()

In [27]:
import json
import plotly.express as px

# Load data from "samples.json"
with open('samples.json') as f:
    data = json.load(f)

sample_data = data['samples']

# Choose a sample to visualize (let's say the first sample)
sample = sample_data[0]

# Extract data for the bubble chart
x_values = sample['otu_ids']
y_values = sample['sample_values']
marker_size = sample['sample_values']
marker_colors = sample['otu_ids']
text_values = sample['otu_labels']

# Create the bubble chart
fig = px.scatter(x=x_values, y=y_values, size=marker_size, color=marker_colors, text=text_values,
                 labels={'x': 'OTU IDs', 'y': 'Sample Values'},
                 title='Bubble Chart of Sample OTU IDs vs. Sample Values',
                 hover_name=text_values)

fig.update_traces(marker=dict(size=marker_size, color=marker_colors), textfont_size=10)

# Show the plot
fig.show()


In [6]:
from IPython.display import display, HTML
import json

# Load the samples data from samples.json
with open('samples.json') as f:
    data = json.load(f)

# Replace 'BB_940' with the desired sample ID
selected_sample_id = 'BB_940'
selected_metadata = next((item for item in data['metadata'] if item['id'] == selected_sample_id), None)

if selected_metadata:
    # Prepare HTML for displaying metadata
    metadata_html = '<table>'
    for key, value in selected_metadata.items():
        metadata_html += f'<tr><td><strong>{key}:</strong></td><td>{value}</td></tr>'
    metadata_html += '</table>'

    # Display metadata
    display(HTML(metadata_html))
else:
    display('Sample ID not found.')



'Sample ID not found.'

In [7]:
import json

# Load data from samples.json
with open('samples.json') as f:
    data = json.load(f)

samples_metadata = data['metadata']


In [8]:
import json

with open('samples.json', 'r') as file:
    samples_data = json.load(file)
    metadata_list = samples_data['metadata']

    for metadata in metadata_list:
        for key, value in metadata.items():
            display(f"{key}: {value}")


'id: 940'

'ethnicity: Caucasian'

'gender: F'

'age: 24.0'

'location: Beaufort/NC'

...

'gender: F'

'age: 22.0'

'location: NC'

'bbtype: I'

'wfreq: 2.0'

In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json

# Load data from samples.json
with open('samples.json') as f:
    samples_data = json.load(f)
    metadata = samples_data['metadata']
    # Add code to process data for plots

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sample Dashboard"),
    html.Div([
        dcc.Dropdown(
            id='sample-dropdown',
            options=[{'label': f"Sample {sample}", 'value': sample} for sample in metadata],
            value=metadata[0]['id']
        ),
    ]),
    # Add code to display metadata and plots
])

@app.callback(
    Output('some-plot', 'figure'),  # Update this with your plot ID
    [Input('sample-dropdown', 'value')]
)
def update_plots(selected_sample):
    # Add code to update plots based on selected_sample
    # Use selected_sample to filter data and update plots

if __name__ == '__main__':
    app.run_server(debug=True)


IndentationError: expected an indented block after function definition on line 31 (2930081354.py, line 35)

In [10]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json

# Load data from samples.json
with open('samples.json') as f:
    samples_data = json.load(f)
    metadata = samples_data['metadata']
    # Add code to process data for plots

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sample Dashboard"),
    html.Div([
        dcc.Dropdown(
            id='sample-dropdown',
            options=[{'label': f"Sample {sample}", 'value': sample} for sample in metadata],
            value=metadata[0]['id']
        ),
    ]),
    # Add code to display metadata and plots
])

@app.callback(
    Output('some-plot', 'figure'),  # Update this with your plot ID
    [Input('sample-dropdown', 'value')]
)
def update_plots(selected_sample):
    # Add code to update plots based on selected_sample
    # Use selected_sample to filter data and update plots

if __name__ == '__main__':
    app.run_server(debug=True)


IndentationError: expected an indented block after function definition on line 31 (2930081354.py, line 35)

In [12]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json

# Load data from samples.json
with open('samples.json') as f:
    samples_data = json.load(f)
    metadata = samples_data['metadata']
    # Add code to process data for plots

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sample Dashboard"),
    html.Div([
        dcc.Dropdown(
            id='sample-dropdown',
            options=[{'label': f"Sample {sample['id']}", 'value': sample['id']} for sample in metadata],
            value=metadata[0]['id']
        ),
    ]),
    # Add code to display metadata and plots
])

@app.callback(
    Output('some-plot', 'figure'),  # Update this with your plot ID
    [Input('sample-dropdown', 'value')]
)
def update_plots(selected_sample):
    # Add code to update plots based on selected_sample
    # Use selected_sample to filter data and update plots

if __name__ == '__main__':
    app.run_server(debug=True)


IndentationError: expected an indented block after function definition on line 31 (3106726423.py, line 35)

In [13]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json

# Load data from samples.json
with open('samples.json') as f:
    samples_data = json.load(f)
    metadata = samples_data['metadata']
    # Add code to process data for plots

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sample Dashboard"),
    html.Div([
        dcc.Dropdown(
            id='sample-dropdown',
            options=[{'label': f"Sample {sample['id']}", 'value': sample['id']} for sample in metadata],
            value=metadata[0]['id']
        ),
    ]),
    # Add code to display metadata and plots
])

@app.callback(
    Output('some-plot', 'figure'),  # Update this with your plot ID
    [Input('sample-dropdown', 'value')]
)
def update_plots(selected_sample):
    # Add code to update plots based on selected_sample
    # Use selected_sample to filter data and update plots



SyntaxError: incomplete input (2195518188.py, line 34)

In [25]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json

# Load data from samples.json
with open('samples.json') as f:
    samples_data = json.load(f)
    metadata = samples_data['metadata']
    # Add code to process data for plots

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sample Dashboard"),
    html.Div([
        dcc.Dropdown(
            id='sample-dropdown',
            options=[{'label': f"Sample {sample['id']}", 'value': sample['id']} for sample in metadata],
            value=metadata[0]['id']
        ),
    ]),
    # Add code to display metadata and plots
])

@app.callback(
    Output('some-plot', 'figure'),  # Update this with your plot ID
    [Input('sample-dropdown', 'value')]
)
def update_plots(selected_sample):
    # Add code to update plots based on selected_sample
    # Use selected_sample to filter data and update plots
    pass  # Placeholder for your code

if __name__ == '__main__':
    app.run_server(debug=True)


In [14]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json

# Load data from samples.json
with open('samples.json') as f:
    samples_data = json.load(f)
    metadata = samples_data['metadata']
    # Add code to process data for plots

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sample Dashboard"),
    html.Div([
        dcc.Dropdown(
            id='sample-dropdown',
            options=[{'label': f"Sample {sample['id']}", 'value': sample['id']} for sample in metadata],
            value=metadata[0]['id']
        ),
    ]),
    # Add code to display metadata and plots
])

@app.callback(
    Output('some-plot', 'figure'),  # Update this with your plot ID
    [Input('sample-dropdown', 'value')]
)
def update_plots(selected_sample):
    # Add code to update plots based on selected_sample
    # Use selected_sample to filter data and update plots
    pass  # Placeholder for your code

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\kyled\AppData\Local\Temp\ipykernel_29004\2678683025.py:2: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

C:\Users\kyled\AppData\Local\Temp\ipykernel_29004\2678683025.py:3: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [26]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json
import plotly.express as px

# Load the samples.json data
with open('samples.json') as file:
    samples_data = json.load(file)

app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Sample Dashboard"),
    
    # Dropdown for sample selection
    dcc.Dropdown(
        id='sample-dropdown',
        options=[{'label': sample['id'], 'value': sample['id']} for sample in samples_data],
        value=samples_data[0]['id']  # Default value
    ),
    
    # Display metadata
    html.Div(id='metadata-output'),
    
    # Display plots
    dcc.Graph(id='plot1'),
    dcc.Graph(id='plot2')
])

# Callback to update metadata and plots based on sample selection
@app.callback(
    [Output('metadata-output', 'children'),
     Output('plot1', 'figure'),
     Output('plot2', 'figure')],
    [Input('sample-dropdown', 'value')]
)
def update_display(selected_sample):
    # Find the selected sample's data
    selected_data = next((sample for sample in samples_data if sample['id'] == selected_sample), None)
    if selected_data is None:
        return "Sample not found", {}, {}
    
    # Extract metadata
    metadata = selected_data['metadata']
    metadata_output = html.Div([html.P(f"{key}: {value}") for key, value in metadata.items()])
    
    # Create plots (modify this part based on your data and plots)
    plot1 = px.bar(x=[1, 2, 3], y=[4, 5, 6], labels={'x': 'X-axis', 'y': 'Y-axis'}, title='Bar Plot')
    plot2 = px.scatter(x=[1, 2, 3], y=[4, 5, 6], labels={'x': 'X-axis', 'y': 'Y-axis'}, title='Scatter Plot')
    
    return metadata_output, plot1, plot2

if __name__ == '__main__':
    app.run_server(debug=True)


TypeError: string indices must be integers